# Compute the polarizability tensor

MyBigDFT comes with some classes implementing particular workflows of calculations. These workflows define a queue of jobs, that can easily be run sequentially, without having to worry about the `Job` context manager. They also generally define a particular post-processing procedure, run after all the BigDFT calculations in order to extract some meaningful imformation.

The example provided here shows how to obtain the polarizability tensor of the N$_2$ molecule by using the [PolTensor](https://mmoriniere.gitlab.io/MyBigDFT/poltensor.html) class. 

The polarizability tensor represents the response of the charges of a system (its dipole) to the application of an external electric field.
To compute this polarizability tensor, some BigDFT calculations are performed, where the system is subject to an external electric field along each direction of space ($x$, $y$ and $z$). The elements of the polarizability tensor are then defined by the ratio of the delta of the dipole in one direction and the delta of the electric field amplitudes:
$\alpha_{ij} = \frac{\Delta D_i}{\Delta E_j}$
where $i, j \in \{x, y, z\}$, $\Delta D_i$ is the variation of the dipole along the $i$ direction and $\Delta E_j$ is the variation of the electric field amplitude along the $j$ direction

## Initialization

You first need to import some useful classes to define a ground state calculation as well as the `PolTensor` class:

In [1]:
from mybigdft import Job, Posinp, Atom
from mybigdft.workflows import PolTensor

You are then able to define an instance of the `PolTensor` class. To that end, you only need to provide a Job defining the ground state calculation. It simply amounts to proving initial positions, some non-default input parameters (if needed) and initializing the job. Default values are used to define the electric field amplitudes applied along each direction, but you may specify them via the `ef_amplitudes` optional argument.

In [2]:
atoms = [Atom('N', [0, 0, 0]), Atom('N', [0, 0, 1.095])]
pos = Posinp(atoms, units='angstroem', boundary_conditions='free')
gs = Job(posinp=pos, name='N2', run_dir='../../../tests/pol_tensor_N2')
pt = PolTensor(gs)
# The line above actually amounts to:
# pt = PolTensor(gs, ef_amplitudes=[1.e-4]*3)

You can then run the workflow without having to care about the context manager to run each calculation:

In [3]:
pt.run()

/Users/maximemoriniere/Documents/Python/MyBigDFT/tests/pol_tensor_N2
Logfile log-N2.yaml already exists!

/Users/maximemoriniere/Documents/Python/MyBigDFT/tests/pol_tensor_N2
Data directory not found for reference calculation.
Logfile log-N2_EF_along_x.yaml already exists!

/Users/maximemoriniere/Documents/Python/MyBigDFT/tests/pol_tensor_N2
Data directory not found for reference calculation.
Logfile log-N2_EF_along_y.yaml already exists!

/Users/maximemoriniere/Documents/Python/MyBigDFT/tests/pol_tensor_N2
Data directory not found for reference calculation.
Logfile log-N2_EF_along_z.yaml already exists!



Note that the calculations were run with different names in the same folder as the one specified by the ground state job: this was taken care of by the PolTensor class.

You can loop over the jobs in the queue to see which were the input parameters used:

In [5]:
for job in pt.queue:
    print(f"{job.name}: {job.inputparams}")

N2: {}
N2_EF_along_x: {'dft': {'elecfield': [0.0001, 0.0, 0.0]}}
N2_EF_along_y: {'dft': {'elecfield': [0.0, 0.0001, 0.0]}}
N2_EF_along_z: {'dft': {'elecfield': [0.0, 0.0, 0.0001]}}


You can finally get the value of the polarizability tensor (in atomic units) via the `pol_tensor` attribute:

In [4]:
pt.pol_tensor

array([[ 1.05558e+01, -2.00000e-04, -2.00000e-04],
       [-2.00000e-04,  1.05558e+01, -2.00000e-04],
       [ 0.00000e+00,  0.00000e+00,  1.50535e+01]])

As expected, the polarizability tensor is diagonal and $\alpha_{xx} = \alpha_{yy} < \alpha_{zz}$. The values are consistent with what is reported in the litterature and the experiment (see Table II of [George Maroulis and Ajit J. Thakkar, *J. Chem. Phys.* **88**, 7623 (1988)](http://dx.doi.org/10.1063/1.454327)). Keep in mind that default input parameters were used, so that the the present calculations cannot be considered as converged.

**Be carefull**: if the electric fields applied are high enough for some electrons to escape the system, then you cannot trust the results anymore. In such a case, the system you are trying to simulate is not the one you expected, because an occupied orbital has a positive energy under the perturbation of the elctric field. There is actually a rule of thumb to know if the electric field applied along a direction is not too high: the difference of potential along the electric field (*i.e.* the amplitude of the electric field times the length of the BigDFT grid along the direction of the electric field) has to be roughly two times smaller than the absolute value of the HOMO when there is no perturbation.